In [1]:
import os, sys, codecs, time, datetime
import fileinput
import os.path
import stanza
from io import StringIO
from subprocess import call


from docopt import docopt
from propsde.applications.viz_tree import DepTreeVisualizer
%load_ext autoreload
%autoreload 2
import propsde.applications.run as run
import sys
if sys.version_info[0] >= 3:
    unicode = str
    
#stanza.download('de')
#nlp = stanza.Pipeline('de')

In [2]:
import pickle

In [3]:
with open('polizeiberichte.pkl', 'rb') as f:
    data = pickle.load(f, encoding="utf8")

Gute Texte: 5
Schwierige Texte: 11

In [16]:
text_example = data[11]["text1"]
print(text_example)

Am Donnerstag, dem 27. Februar gegen 10:30 Uhr, hatten zunächst zwei Männer das Juweliergeschäft „Stöger“ in der Berkaer Straße 6 im Ortsteil Schmargendorf betreten. Sie drängten den 61-jährigen Mitinhaber unter Drohung mit einer Schusswaffe in die hinteren Räume, brachten ihn zu Boden, schlugen mit dem Pistolengriff mehrmals auf seinen Kopf ein und fesselten ihn anschließend. Fast gleichzeitig betrat ein Komplize mit einer Sporttasche das Geschäft. Die drei Täter rafften nunmehr aus den Vitrinen und Auslagen hochwertigen Schmuck sowie insgesamt fünfzehn Armbanduhren zusammen und flüchteten anschließend in unbekannte Richtung.


In [4]:
file = codecs.open("examples.txt", encoding='utf8')
gs = run.parseSentences(file)

In [6]:
for graph, tree in gs:
    original_line = str(graph).split("\n")[0].split(" ")
    

In [78]:
for graph, tree in gs:
    for node in graph:
        if node.coreference != None:
            print(node.text[0], node.coreference)
            print(node.coreference)

Zwei (0
(0
winzig 0
0
kleine 0
0
Männer 0)
0)
Sie (0)
(0)


# ParZU Dependency Parser

In [8]:
import urllib.request
import urllib.parse

def parse_dependencies_to_conll_file(text, path):
    params = {
        "text": text,
        "format": "conll"
    }
    args = urllib.parse.urlencode(params).encode("utf-8")
    req = "http://localhost:5003/parse/?"+args.decode("utf-8")
    try:
        f = urllib.request.urlopen(req)
        if path:
            file = open(path,'w')
            file.write(f.read().decode('utf-8'))
        else:
            print(f.read().decode('utf-8'))
        return 0
    except Exception as ex:
        print(ex)
        return 1
    
parse_dependencies_to_conll_file("Alex sagt Chris", "test2.conll")

0

# CorZu 2.0

In [70]:
import os
output = "corzu_export"
conll = "test.conll"
markables = "markables"
text = open("examples.txt", "r").read()

text_example = "Zwei winzig kleine Männer essen Kuchen. Sie mögen Kuchen gerne."

err = parse_dependencies_to_conll_file(text_example, conll)
if err == 0:
    cmd = "python ext/CorZu_v2.0/extract_mables_from_parzu.py " + conll + " > " + markables
    err = os.system(cmd)
    print(err)



0


In [71]:
cmd = "python ext/CorZu_v2.0/corzu.py " + markables + " " + conll + " " + output    
err = os.system(cmd)
print(err)

0


In [12]:
parse_dependencies_to_conll_file("Paul sagt zu Erik, dass er den Schlüssel vergessen hat", None)
parse_dependencies_to_conll_file("Paul sagt zu Erik, dass er zurück gehen sollte", None)


1	Paul	Paul	N	NE	Masc|Nom|Sg	2	subj	_	_
2	sagt	sagen	V	VVFIN	3|Sg|Pres|Ind	0	root	_	_
3	zu	zu	PREP	APPR	Dat	2	pp	_	_
4	Erik	Erik	N	NE	_|Dat|_	3	pn	_	_
5	,	,	$,	$,	_	0	root	_	_
6	dass	dass	KOUS	KOUS	_	11	konj	_	_
7	er	er	PRO	PPER	3|Sg|Masc|Nom	11	subj	_	_
8	den	die	ART	ART	Def|Masc|Acc|Sg	9	det	_	_
9	Schlüssel	Schlüssel	N	NN	Masc|Acc|Sg	10	obja	_	_
10	vergessen	veressen	V	VVPP	_	11	aux	_	_
11	hat	haben	V	VAFIN	3|Sg|Pres|Ind	2	objc	_	_


1	Paul	Paul	N	NE	Masc|Nom|Sg	2	subj	_	_
2	sagt	sagen	V	VVFIN	3|Sg|Pres|Ind	0	root	_	_
3	zu	zu	PREP	APPR	Dat	2	pp	_	_
4	Erik	Erik	N	NE	_|Dat|_	3	pn	_	_
5	,	,	$,	$,	_	0	root	_	_
6	dass	dass	KOUS	KOUS	_	0	root	_	_
7	er	er	PRO	PPER	3|Sg|Masc|Nom	0	root	_	_
8	zurück	zurück	PTKVZ	PTKVZ	_	0	root	_	_
9	gehen	gehen	V	VVINF	_	10	aux	_	_
10	sollte	sollen	V	VMFIN	_|Sg|Past|_	0	root	_	_




0

# Work with CorZu Extraction

In [17]:
import re

def find_np(lines, id):
    np = ""
    indices = 0
    for line in lines:
        line_list = re.split('\t| +',line)
        np += line_list[1] + " "
        indices += 1
        if line.strip().endswith(id + ')'):
            return np, indices


text = open(output, "r").read()
lines=text.split('\n\n')

for i in range(0, len(lines)):
    lines[i] = lines[i].split("\n")
    for j in range(0, len(lines[i])):
        lines[i][j] = lines[i][j].split("\t")
lines = {i: l for i, l in enumerate(lines) if len(l) > 0}


for sentence_count, gt in enumerate(gs):
    graph, tree = gt
    word_ids = []
    for node in graph.nodes():
        index = int(node.to_conll_like().split("\t")[1].split(",")[0]) -1
        if lines[sentence_count][index][-1] != "-" and lines[sentence_count][index][-1] != "":
            node.features["coreference"] = lines[sentence_count][index][-1]


NameError: name 'output' is not defined

In [18]:
from graphviz import Source

i = 0
for graph, tree in gs:
    dot = graph.writeToDot(None, None)

    s = Source(dot, filename=str(i) + "_test.gv", format="png")
    s.view()
    i += 1
    break

# NEO4J Connection

In [5]:
from neo4j import GraphDatabase


def _create_node(tx, uid, label, coreference):
    tx.run("CREATE (e:Node {label: $label, uid: $uid, coreference: $coreference}) RETURN e", label=label, uid=uid, coreference=coreference)
    
def _create_edge(tx, src, dst, label):
    tx.run("MATCH (src:Node), (dst:Node) WHERE src.uid = $src AND dst.uid = $dst MERGE (src)-[r:edge {label: $label}]->(dst)", src=src, dst=dst, label=label)
    
def _delete_all(tx):
    tx.run("MATCH (n) DETACH DELETE n")

def create_node(driver, uid, label, coreference=None):
    with driver.session() as session:
        session.write_transaction(_create_node, uid, label, coreference)
            
def create_edge(driver, src, dst, label):
    with driver.session() as session:
        session.write_transaction(_create_edge, src, dst, label)
    
def delete_all(driver):
    with driver.session() as session:
        session.write_transaction(_delete_all)
        

In [44]:
from pygraph.classes.digraph import digraph
from collections import defaultdict
import re

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "852963"))

coreference_map = defaultdict(list)

coreference_head = {}

coreference_id_list = []

def _add_coreference_heads_for_graph(graph, coreference_heads):
    for u_v in graph.edges():
        no_coreference = False
        for edge in u_v:
            if not edge.coreference:
                no_coreference = True
            elif no_coreference:
                if edge.coreference:
                    coref_id = int(re.findall(r'\d+', edge.coreference)[0])
                    if coref_id not in coreference_heads:
                        coreference_heads[coref_id] = edge.uid
                    elif edge.uid < coreference_heads[coref_id]:
                        coreference_heads[coref_id] = edge.uid
    return coreference_heads
    

def _is_part_of_initial_coreference(nodes, uid):
    current_node = nodes[uid]
    if current_node.coreference:
        coref_id = int(re.findall(r'\d+', current_node.coreference)[0])

        if coref_id not in coreference_map:
            coreference_map[coref_id].append(current_node.coreference)
            return True
        else:
            is_finished = False
            for coref in coreference_map[coref_id]:
                if coref[-1] == ")":
                    is_finished = True
            if not is_finished:
                if current_node.coreference:
                    coreference_map[coref_id].append(current_node.coreference)
                return True
            else:
                return False
    
                    
def write_graphs_to_neo4j(driver, graphs):
    delete_all(driver)
    coreference_heads = {}
    for graph, tree in graphs:
        nodes = graph.nodesMap
        _add_coreference_heads_for_graph(graph, coreference_heads)
    
        for uid in nodes:
            node = nodes[uid]
            # NODE
            label = str(node.text[0])
            if node.isPredicate:
                # is predicate
                pass
            if node.features.get("top", False):
                # is top
                pass
            
            # Remove Punctuation Marks
            if len(label) > 1:
                if node.coreference and _is_part_of_initial_coreference(nodes, uid):
                    create_node(driver, uid, label, node.coreference)   
                elif not node.coreference:
                    create_node(driver, uid, label, node.coreference)
            
            # EDGE
            for (src, dst) in digraph.edges(graph):
                
                label = str(graph.edge_label((src, dst)))
                if label:
                    coref_id = None
                    if nodes[src].coreference:
                        coref_id = int(re.findall(r'\d+', nodes[dst].coreference)[0])
                        create_edge(driver, coreference_heads.get(coref_id, src), dst, label) 
                    elif nodes[dst].coreference:
                        coref_id = int(re.findall(r'\d+', nodes[dst].coreference)[0])
                        create_edge(driver, src, coreference_heads.get(coref_id, dst), label) 
                    else:
                        create_edge(driver, src, dst, label) 

    
#delete_all(driver)
write_graphs_to_neo4j(driver, gs)
driver.close()

In [156]:
for graph, tree in gs:
    print(graph)

Am Donnerstag , dem 27. Februar gegen 10 : 30 Uhr , hatten zunächst zwei Männer das Juweliergeschäft „ Stöger “ in der Berkaer Straße 6 im Ortsteil Schmargendorf betreten .
0	1,Am	APPRART	0	1	
1	2,Donnerstag	NN	0	0	pn,0
2	3,,	$,	0	1	
3	4,dem	ART	0	0	det,5
4	5,27.	ADJA	0	0	attr,5
5	6,Februar	NN	0	0	app,1
6	7,gegen	APPR	0	0	pp,5
7	8,10	CARD	0	0	pn,6
8	9,:	$.	0	1	
9	10,30	CARD	0	0	attr,10
10	11,Uhr	NN	0	0	vok,12
11	12,,	$,	0	1	
12	13,hatten	VAFIN	1	1	
13	14,zunächst	ADV	0	0	adv,12
14	15,zwei	CARD	0	0	attr,15
15	16,Männer	NN	0	0	subj,12
16	17,das	ART	0	0	det,17
17	18,Juweliergeschäft	NN	0	0	obja,12
18	19,„	NE	0	0	app,17
19	20,Stöger	NE	0	0	app,18
20	21,“	$(	0	1	
21	22,in	APPR	0	0	pp,29
22	23,der	ART	0	0	det,23
23	24,Berkaer	NN	0	0	pn,21
24	25,Straße	NN	0	0	obja,29
25	26,6	CARD	0	0	app,24
26	27,im	APPRART	0	0	pp,29
27	28,Ortsteil	NN	0	0	pn,26
28	29,Schmargendorf	NE	0	0	app,27
29	30,betreten	VVINF	1	1	
30	31,.	$.	0	1	

Sie drängten den 61-jährigen Mitinhaber unter Drohung mit einer Schusswaf

In [39]:
sentence_index = 0
for graph, tree in gs:
    sentences[sentence_index] = {}
    for line in lines:
            l = line.split('\t')
            line_list = re.split('\t| +',line)

            id = re.findall('\d+',line_list[-1])
            if id and id[0] not in entities and line.strip().endswith('(' + id[0]):
    
                sentences[sentence_index][l[0]] = id[0]
                print(i, l[0], l[1])
                entities[i] = "(" + id[0]
    
                for j in range(1, indices-2):
         
                    entities[i + j] = "-"
                entities[i + indices] = id[0] + ")"
    
            elif id and line.strip().endswith('(' + id[0] + ')'):
                entities[i] = "(" + id[0] + ")"
    sentence_index += 1

NameError: name 'sentences' is not defined

# Combining CorZu and PropsDE

In [41]:
#DONE WITH CORZU AS INPUT FILE
            

### Extract propositions with words and their ids

In [18]:
from functools import reduce
from itertools import product
from propsde.graph_representation.graph_utils import get_min_max_span, find_nodes, \
    find_edges, merge_nodes, multi_get, duplicateEdge, accessibility_wo_self, \
    replace_head, find_marker_idx
from propsde.utils.utils import encode_german_characters, encode_german_chars
from propsde.graph_representation.proposition import Proposition
from propsde.graph_representation.word import NO_INDEX, Word, strip_punctuations

def subgraph_to_string(graph,node,exclude=[]):
    nodes = [node]
    change = True
    while change:
        change=False
        for curNode in nodes:
            for curNeigbour in graph.neighbors(curNode):
                if (curNeigbour in nodes) or (curNeigbour in exclude): continue
                nodes.append(curNeigbour)
                change = True
    #print [[w.word for w in x.text] for x in nodes]

#     minInd = min([n.minIndex() for n in nodes])-1
#     maxInd = max([n.maxIndex() for n in nodes])-1
#     ret = " ".join(graph.originalSentence.split(" ")[minInd:maxInd+1])
#     nodes = [n for n in minimal_spanning_tree(graph, node) if not n in exclude]
#     ret = " ".join(node.get_original_text() for node in sorted(nodes,key = lambda n: n.minIndex()))
    try:
        ret = ""
        words = []
        ids = []
        for n in nodes:
            words += n.surface_form
            ids.append(n.uid)
            # add collapsed prepositions
            for parent_rel in n.incidents():
                # but just nested ones
                if parent_rel.startswith("prep_") and n.incidents()[parent_rel][0] in nodes:
                    idx = get_min_max_span(graph,n)[0] - 1
                    w = Word(idx,parent_rel.replace("prep_",""))
                    words += [w]    

        words = list(set(words))
        for w,i  in zip(words, ids):
            print(w, i)
        ret = " ".join([w.word for w in strip_punctuations(sorted(words,key=lambda w:w.index))])+" "

    except:
        raise Exception()
#     
    
    return "schwub"

def getPropositions(self):
    ret = []
    for topNode in [n for n in self.nodes() if n.features.get("top", False)]:
            #print [w.word for w in topNode.text]
#            if "dups" in topNode.features:
            dups = topNode.features.get("dups", [])
            allDups = reduce(lambda x, y:list(x) + list(y), dups, [])

            rest = [n for n in self.neighbors(topNode) if n not in allDups]
            neigboursList = []
            for combination in product(*dups):
                curNeigbourList = []
                ls = list(combination) + rest
                for curNode in ls:
                    curNeigbourList.append((self.edge_label((topNode, curNode)), curNode))
                neigboursList.append(curNeigbourList)
         
            for nlist in neigboursList:
                #print [[w.word for w in x[1].text] for x in nlist]
                argList = []
                all_neighbours = [n for _, n in nlist]
                for k, curNeighbour in sorted(nlist, key=lambda k_n:get_min_max_span(self, k_n[1])[0]):
                    curExclude = [n for n in all_neighbours if n != curNeighbour] + [topNode]
                    if self.edge_label((topNode,curNeighbour)) != "dep":
                        
                        argList.append([k, subgraph_to_string(self, curNeighbour, exclude=curExclude), curNeighbour.uid])
   
                if topNode.features.get("Lemma"):
                    topNodeText = encode_german_characters(topNode.features.get("Lemma"))
                else:
                    topNodeText = topNode.get_original_text()
                if topNode.features.get("Negation",False):
                    topNodeText = "nicht " + topNodeText
                argList = [a for a in argList if not a[1].strip() in ["und","oder"]]
                
                if not len(argList) == 0:
                    ret.append((topNode, argList))
    return ret


In [160]:
word_indices = {}
ready = {}
current_length = 0
predicates = []

for g, tree in gs:     
    prop = getPropositions(g)
    print(prop)

NameError: name 'getPropositions' is not defined

In [21]:
from itertools import compress

def _extract_subj_from_article_stanza(pos_tag, personal_pronouns, pronoun):
    for sent in pos_tag.sentences:
        for word in sent.words:
            if word.deprel == "nsubj" or word.deprel == "root":
                if callable(personal_pronouns[pronoun]):
                    _pronoun = personal_pronouns[pronoun](word.text)
                else:
                    _pronoun = personal_pronouns[pronoun]
                return (_pronoun, word.lemma)
            
def _get_pronoun_for_den(word):
    if word[-2:] == "en":
        return "sie"
    else:
        return "er"

def _map_article_of_subj_to_pronoun(arg):
    personal_pronouns = {"der": "er", "das": "es", "die": "sie", "dem": "er", "den": _get_pronoun_for_den}
    
    if "subj" in arg:
        words = arg[1].lower().split()
        pronoun_place = list(word in personal_pronouns for word in words)
        pronoun = list(compress(words, pronoun_place))
    
        if len(pronoun) == 1:
            pos_tag = nlp(arg[1])
            return _extract_subj_from_article_stanza(pos_tag, personal_pronouns, pronoun[0])
    
    for pronoun in personal_pronouns:
        if pronoun in arg[1] and "prep" not in arg[0]:
            pos_tag = nlp(arg[1])
            return _extract_subj_from_article_stanza(pos_tag, personal_pronouns, pronoun)
                        
    return None

def _get_core_from_noun_phrase(noun_phrase):
    pos_tag = nlp(noun_phrase)
    for sent in pos_tag.sentences:
        for word in sent.words:
            if word.head == 0:
                return {"np": noun_phrase, "core":word.lemma}
            
def _get_context(prop_cache, subjects, words):
    for j in range(0, len(subjects)):
        for k in range(len(subjects[index-j]) - 1, 0, -1):
            if index > 0 and len(subjects[index-j]) > 0 and subjects[index-j][k][0] in words:
                subject_index = words.index(subjects[index-j][k][0])
                return _get_core_from_noun_phrase(subjects[index-j][k][1])["core"]
            
def pretty_print(prop_cache):
    for prop in prop_cache:
        test = {}
        for arg in prop["args"]:
            if "context" in arg[1]:
                test[arg[0]] = arg[1]["context"]
            else:
                test[arg[0]] = arg[1]["core"]
        prep_exists = False
        for k in test:
            if "_" in k:
                if "conj" in k:
                    pass
                else:
                    if "subj" in k:
                        print(test["subj"] + " " + prop["pred"] + " " + k + " " + test[k])
                prep_exists = True
        if not prep_exists:  
            print(test["subj"] + " " + prop["pred"] + " " + test["dobj"])
        
            



In [22]:
subjects = []
prop_cache = []
index = 0

for i, (g, tree) in enumerate(gs):
    for prop in g.getPropositions('pdf'):
        subjects.append([])
        prop_cache.append({"args": prop.args, "pred": prop.pred})
        
while index < len(prop_cache):
    for i, arg in enumerate(prop_cache[index]["args"]):
        words = arg[1].lower().split()

        _article_pronoun = _map_article_of_subj_to_pronoun(arg)
        _noun_phrase_core = _get_core_from_noun_phrase(arg[1])
        if _article_pronoun:
            subjects[index].append(_article_pronoun)
        if _noun_phrase_core:
            prop_cache[index]["args"][i][1] = _noun_phrase_core
        context = _get_context(prop_cache, subjects, words)
        if context:
            prop_cache[index]["args"][i][1]["context"] = context

                    

    index += 1
pretty_print(prop_cache)
print(subjects)

[]


Der regierende Bürgermeister fährt zum Rathaus.
Im roten Rathaus geht er in den Sitzungssaal.
Der alte Sitzungssaal liegt neben dem Büro von Michael Müller.

In [158]:
for prop in prop_cache:
    str_build = ""
    
    for arg in prop["args"]:
        if arg[0] is not "mod":
            if "context" in arg[1]:
                str_build += arg[1]['context'] + "/"

            str_build += arg[1]['core'] + "/"
    print(prop["pred"] + ": " + str_build)

NameError: name 'prop_cache' is not defined

versuchen:(subj:Die Täter , prep_in:der Nacht vom 19 )
Juni 2014:(dobj:die Eingangstür des Bürokomplexes in der Feldtmannstraße gewaltsam zu öffnen )
gelangt:(subj:sie , prep_in:das Objekt , mod:so )
aufhebeln:(subj:sie , dobj:ein Fenster an der Rückseite des Hauses , mod:Als dies nicht gelang , mod:schließlich )
und:(conj_und:Als dies nicht gelang , hebelten sie schließlich ein Fenster an der Rückseite des Hauses auf , conj_und:sie gelangten so in das Objekt )
entwenden:(subj:sie , dobj:auch eine EC-Karte , die bereits an Freitag , 20 , prep_aus:den Büroräumen mehrerer dort ansässiger Firmen , prep_neben:Geld )
20:(subj:auch eine EC-Karte , prep_an:Freitag , mod:bereits )
einsetzen:(obj:Juni 2014 , prep_gegen:5.10 Uhr , prep_an:einem Geldautomaten in der Danziger Straße )

In [504]:
for prop in prop_cache:
    for arg in prop["args"]:
        print(arg[1])

{'np': 'Gestern ', 'core': 'Gestern'}
{'np': 'Nachmittag ', 'core': 'Nachmittag'}
{'np': 'einem schweren Verkehrsunfall in Rudow ', 'core': 'Verkehrsunfall'}
{'np': '17:30 Uhr ', 'core': 'Uhr'}
{'np': 'ein Opel ', 'core': 'Opel'}
{'np': 'der Schönefelder Straße in Richtung Rudower Spinne ', 'core': 'Straße'}
{'np': 'einen Mercedes ', 'core': 'Mercedes'}
{'np': 'einen Mercedes ', 'core': 'Mercedes'}
{'np': 'der Neuköllner Straße in Richtung Waltersdorfer Chaussee ', 'core': 'Straße'}
{'np': 'unterwegs ', 'core': 'unterwegs'}
{'np': 'Als die Fahrerin die Kreuzung Neuköllner Straße passieren wollte ', 'core': 'passieren'}
{'np': 'den ersten Ermittlungen zufolge ', 'core': 'Ermittlung'}
{'np': 'sie ', 'core': 'sie', 'context': 'Ermittlung'}
{'np': 'einen Mercedes ', 'core': 'Mercedes'}
{'np': 'einem Zusammenstoß ', 'core': 'Zusammenstoß'}
{'np': 'Der Mercedes ', 'core': 'Mercedes'}
{'np': 'Schleudern ', 'core': 'schleudern'}
{'np': 'Die Feuerwehr ', 'core': 'Feuerwehr'}
{'np': 'den 49 Jahr

In [494]:
nlp('Die Feuerwehr brachte ihn zu stationären Behandlung in ein Krankenhaus ')

[
  [
    {
      "id": "1",
      "text": "Die",
      "lemma": "der",
      "upos": "DET",
      "xpos": "ART",
      "feats": "Case=Nom|Definite=Def|Gender=Fem|Number=Sing|PronType=Art",
      "head": 2,
      "deprel": "det",
      "misc": "start_char=0|end_char=3"
    },
    {
      "id": "2",
      "text": "Feuerwehr",
      "lemma": "Feuerwehr",
      "upos": "NOUN",
      "xpos": "NN",
      "feats": "Case=Nom|Gender=Fem|Number=Sing",
      "head": 3,
      "deprel": "nsubj",
      "misc": "start_char=4|end_char=13"
    },
    {
      "id": "3",
      "text": "brachte",
      "lemma": "bringen",
      "upos": "VERB",
      "xpos": "VVFIN",
      "feats": "Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin",
      "head": 0,
      "deprel": "root",
      "misc": "start_char=14|end_char=21"
    },
    {
      "id": "4",
      "text": "ihn",
      "lemma": "er",
      "upos": "PRON",
      "xpos": "PPER",
      "feats": "Case=Acc|Gender=Masc|Number=Sing|Person=3|PronType=Prs",


1	was	was	PRO	PIS	_|Nom|Sg	2	subj	_	_
2	geht	gehen	V	VVFIN	3|Sg|Pres|Ind	0	root	_	_
3	ab	ab	PTKVZ	PTKVZ	_	2	avz	_	_
4	.	.	$.	$.	_	0	root	_	_


